In [1]:
import torch
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers.image_utils import load_image
from donut import DonutModel

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

pretrained_model = DonutModel.from_pretrained("naver-clova-ix/donut-base")

if torch.cuda.is_available():
    pretrained_model.half()
    pretrained_model.to("cuda")

pretrained_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at naver-clova-ix/donut-base were not used when initializing DonutModel: ['encoder.model.norm.weight', 'encoder.model.norm.bias']
- This IS expected if you are initializing DonutModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS

DonutModel(
  (encoder): SwinEncoder(
    (model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (0): BasicLayer(
          dim=128, input_resolution=(640, 480), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((128,), ep

In [2]:
import donut
json_evalutor = donut.JSONParseEvaluator()

def calc_val_edit(image, label):
    output = pretrained_model.inference(image=image, prompt=f"<s_synthetic_data2>")["predictions"][0]
    score = json_evalutor.cal_acc(output, label)
    return (score, output)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("test")

In [4]:
import numpy

scores = []
predictions = []
ground_truths = []

for ds in dataset["train"]:
    score, prediction = calc_val_edit(ds["image"], ds["text"])
    scores.append(score)
    predictions.append(prediction)
    ground_truths.append(ds["text"])
    
scores = {
    "ted_accuracy": numpy.mean(scores),
    "f1_accuracy": json_evalutor.cal_f1(predictions, ground_truths),
}
scores

{'ted_accuracy': 0.0, 'f1_accuracy': 0.0}

In [5]:
import os
import json
import extract_receipt as er

base_dir = 'testdata_with_labels_1'
acc_list = []
predictions = []
ground_truths = []

for img_name in os.listdir(f'{base_dir}/receipts'):
  if not '.jpg' in img_name:
    continue

  file_num = img_name.replace('.jpg', '')
  image = Image.open(f'{base_dir}/receipts/{img_name}').convert("RGB")
  try:
   image = er.extract_receipt(image)
  except:
    pass
  with open(f'{base_dir}/labels/{file_num}.json', 'r') as f:
     label = json.loads(f.read())

  score, prediction = calc_val_edit(image, label)
  acc_list.append(score)
  predictions.append(prediction)
  ground_truths.append(label)

scores = {
    "ted_accuracy": numpy.mean(acc_list),
    "f1_accuracy": json_evalutor.cal_f1(predictions, ground_truths),
}
scores

{'ted_accuracy': 0.0, 'f1_accuracy': 0.0}